<h2>Implementing Regression Trees</h2>

Import a few packages that you will need. In addition, you will load two binary classification dataset - the spiral dataset and the <a href="https://archive.ics.uci.edu/ml/datasets/Ionosphere">ION</a> dataset.

In [ ]:
import math
from scipy.stats import mode
from google.colab import files
import io
import numpy as np
import pandas as pd
from pylab import *
from numpy.matlib import repmat
import matplotlib
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time
import sys

%matplotlib inline

from helper import *

print('You\'re running python %s' % sys.version.split(' ')[0])

You're running python 3.10.12


The code below generates spiral data using the trigonometric functions sine and cosine, then splits the data into train and test segments.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/Dataset.csv')
df['diagnosis'] = df['diagnosis'].replace({'M': int('0')})
df['diagnosis'] = df['diagnosis'].replace({'B': int('2')})
df.shape

(569, 33)

In [ ]:
Xrawdata, Yrawdata = df.shape
trpercent = 0.8

In [ ]:
dfxTr = df.iloc[:math.floor(Xrawdata*trpercent),2:32]
dfyTr = df.iloc[:math.floor(Xrawdata*trpercent),1:2]
dfxTe = df.iloc[math.floor(Xrawdata*trpercent):Xrawdata,2:32]
dfyTe = df.iloc[math.floor(Xrawdata*trpercent):Xrawdata,1:2]

In [ ]:
xTr = dfxTr.to_numpy()
yTr = dfyTr.to_numpy()
xTe = dfxTe.to_numpy()
yTe = dfyTe.to_numpy()

### Part One: Implement `sqimpurity` [Graded]

First, implement the function **`sqimpurity`**, which takes as input a vector $y$ of $n$ labels and outputs the corresponding squared loss impurity:
$$
\sum_{i = 1}^{n} \left( y_i - \overline{y} \right)^2 \textrm{, where } \overline{y} = \frac{\sum_{i=1}^{n} y_i}{n}.
$$

Again, the squared loss impurity works fine even though our final objective is classification. This is because the labels are binary and classification problems can be framed as regression problems.

In [ ]:
def sqimpurity(yTr):
    """
    Computes the squared loss impurity (variance) of the labels.

    Input:
        yTr: n-dimensional vector of labels

    Output:
        squared loss impurity: weighted variance/squared loss impurity of the labels
    """

    N = np.size(yTr)
    assert N > 0 # must have at least one sample
    impurity = 0

    # YOUR CODE HERE
    #raise NotImplementedError()
    impurity = np.sum((yTr - 1 / N * np.sum(yTr))**2)
    return impurity

Let's plot the shape of the impurity function. We vary the mixture of labels in a set of $n$ labels and calculate the impurity of the labels. When the labels are mostly the same, the impurity should be low. When the labels are evenly split between $+1$ and $-1$, the impurity should be the highest.

### Part Two: Implement `sqsplit` [Graded]

Now implement **`sqsplit`**, which takes as input a data set of size $n \times d$ with labels and computes the best feature and the threshold/cut of the optimal split based on the squared loss impurity. The function outputs a feature dimension `0 <= feature < d`, a cut threshold `cut`, and the impurity loss `bestloss` of this best split.

Recall in the CART algorithm that, to find the split with the minimum impurity, you iterate over all features and cut values along each feature. We enforce that the cut value be the average of the two consecutive data points' feature values.

You should calculate the impurity of a node of data $S$ with two branches $S_L$ and $S_R$ as:
$$
\begin{align*}
I(S) &= \frac{\left| S_L \right|}{|S|} I \left( S_L \right) + \frac{\left| S_R \right|}{|S|} I \left( S_R \right)\\
&= \frac{1}{|S|}\sum_{(\mathbf{x}, y) \in S_L} \left( y-\overline{y}_{S_L} \right)^2 + \frac{1}{|S|} \sum_{(\mathbf{x}, y) \in S_R} \left( y - \overline{y}_{S_R} \right)^2\\
&\propto \sum_{(\mathbf{x}, y) \in S_L} \left( y-\overline{y}_{S_L} \right)^2 + \sum_{(\mathbf{x}, y) \in S_R} \left( y - \overline{y}_{S_R} \right)^2
\end{align*}
$$

**Implementation Notes:**
- For calculating the impurity of a node, you should just return the sum of left and right impurities instead of the average.
- Returned `feature` must be 0-indexed as is consistent with programming in Python.
- If along a feature $f$, two data points $\mathbf{x}_i$ and $\mathbf{x}_j$ have the same value, avoid splitting between them; move to the next pair of data points.

For example, with the following `xTr` of size $4 \times 3$ and `yTr` for 4 points:
$$
\begin{bmatrix}
1 & 0 & 2\\
2 & 0 & 1\\
0 & 0 & 1\\
2 & 1 & 2
\end{bmatrix},
\begin{bmatrix}
1\\1\\1\\-1
\end{bmatrix}
$$
among possible features `[0, 1, 2]`, the best split would be at `feature = 1` and `cut = (0 + 1) / 2 = 0.5`.

<hr>

If you're stuck, we recommend that you start with the naïve algorithm for finding the best split, which involves a double loop over all features `0 <= f < d` and all cut values `xTr[0, f] < (xTr[i, f] + xTr[i+1, f]) / 2 < xTr[n-1, f]` (with `xTr` sorted along feature `f`). This algorithm thus calculates impurities for `d(n-1)` splits. Here's the pseudocode:

<center><img src="cart-id3_best_split_pseudocode.png" width="75%" /></center>

In [ ]:
def sqsplit(xTr, yTr):
    """
    Finds the best feature, cut value, and impurity for a split of (xTr, yTr) based on squared loss impurity.

    Input:
        xTr: n x d matrix of data points
        yTr: n-dimensional vector of labels

    Output:
        feature:  index of the best cut's feature (keep in mind this is 0-indexed)
        cut:      cut-value of the best cut
        bestloss: squared loss impurity of the best cut
    """
    n, d = xTr.shape
    assert d > 0 # must have at least one dimension
    assert n > 1 # must have at least two samples

    bestloss = np.inf
    feature = np.inf
    cut = np.inf

    for col in range(d):
        sort = xTr[:, col].argsort()
        sortx = xTr[sort, col]
        sorty = yTr[sort]

        for row in range(n):
            # if row + 1 exceeds range
            if row + 1 == n:
                continue
            # check where values change
            if sortx[row + 1] > sortx[row]:
                Sl = sorty[:row+1]
                Sr = sorty[row+1:]

                loss = sqimpurity(Sl) + sqimpurity(Sr)

                # if loss is smaller
                if loss < bestloss:
                    bestloss = loss
                    feature = col
                    cut = (sortx[row] + sortx[row + 1]) / 2

    return feature, cut, bestloss

### Part Three: Implement `cart` [Graded]

In this section, you will implement the function **`cart`**, which returns a regression tree based on the minimum squared loss splitting rule. You should use the function `sqsplit` to make your splits.

**Implementation Notes:**
We've provided a tree structure in the form of `TreeNode` for you that can be used for both leaves and nodes. To represent the leaves, you would set all fields except `prediction` to `None`.

Non-leaf nodes will have non-`None` fields for all except `prediction`:
1. `left`: node describing left subtree
2. `right`: node describing right subtree
3. `feature`: index of feature to cut (0-indexed as returned by `sqsplit`)
4. `cut`: cutoff value $t$ ($\leq t$: left and $> t$: right)
5. `prediction`: `None`

In [ ]:
class TreeNode(object):
    """
    Tree class.

    (You don't _need_ to add any methods or fields here but feel
    free to if you like. The tests will only reference the fields
    defined in the constructor below, so be sure to set these
    correctly.)
    """

    def __init__(self, left, right, feature, cut, prediction):
        # Check that all or no arguments are None
        node_or_leaf_args = [left, right, feature, cut]
        assert all([arg == None for arg in node_or_leaf_args]) or all([arg != None for arg in node_or_leaf_args])

        # Check that all None <==> leaf <==> prediction not None
        # Check that all non-None <==> non-leaf <==> prediction is None
        if all([arg == None for arg in node_or_leaf_args]):
            assert prediction is not None
        if all([arg != None for arg in node_or_leaf_args]):
            assert prediction is None

        self.left = left
        self.right = right
        self.feature = feature
        self.cut = cut
        self.prediction = prediction

Now implement the function `cart` using **recursion** (you call `cart` on the left and right subtrees inside the `cart` function). Recall the pseudocode for the CART algorithm.

**NOTE:** In this implementation, you will be using **`np.mean`** for `prediction` argument. To check that floating point values in `xTr` are the same or not, you can use `np.isclose(xTr, xTr[0])`, which returns a list of `True` and `False` based on how different the rows of `xTr` are from the vector `xTr[0]`.

<center><img src="cart-id3_pseudocode.png" width="75%" /></center>

In [ ]:
def cart(xTr, yTr):
    """
    Builds a CART tree.

    Input:
        xTr:      n x d matrix of data
        yTr:      n-dimensional vector

    Output:
        tree: root of decision tree
    """
    n, d = xTr.shape
    node = None

    # YOUR CODE HERE
    #raise NotImplementedError()
    prediction = np.mean(yTr)
    index = np.arange(n)

    # Base Case #1
    if np.all(yTr == yTr[0]):
        tree = TreeNode(None, None, None, None, yTr[0])
    elif np.all(np.isclose(xTr, xTr[0])):
        tree = TreeNode(None, None, None, None, prediction)
    else:
        feature, cut, loss = sqsplit(xTr, yTr)
        left_index = index[xTr[:, feature] <= cut]
        right_index = index[xTr[:, feature] > cut]
        left_xTr = xTr[left_index, :]
        right_xTr = xTr[right_index, :]
        left_yTr = yTr[left_index]
        right_yTr = yTr[right_index]

        left_node = cart(left_xTr, left_yTr)
        right_node = cart(right_xTr, right_yTr)
        tree = TreeNode(left_node, right_node, feature, cut, None)
    return tree

### Part Four: Implement `evaltree` [Graded]

Implement the function **`evaltree`**, which evaluates a decision tree on a given test data set. You essentially need to traverse the tree until you end up in a leaf, where you return the `prediction` value of the leaf. Like the `cart` function, you can call `evaltree` on the left subtree and right subtree on testing points that fall in the corresponding subtrees.

Here's some inspiration:
1. If the `tree` is a leaf, i.e. the left and right subtrees are `None`, return `tree.prediction` for all `m` testing points.
2. If the `tree` is non-leaf, using `tree.feature` and `tree.cut` find testing points with the feature value less than/equal to the threshold and greater than. Now, you can call `evaltree` on `tree.left` and the left set of testing points to obtain the left set's predictions. Then obtain the predictions for the right set, and return all predictions.

In [ ]:
def evaltree(tree, xTe):
    """
    Evaluates testing points in xTe using decision tree root.

    Input:
        tree: TreeNode decision tree
        xTe:  m x d matrix of data points

    Output:
        pred: m-dimensional vector of predictions
    """
    m, d = xTe.shape
    preds = np.zeros(m)

    # YOUR CODE HERE
    #raise NotImplementedError()

    for x in range(m):
        rt = tree
        while(True):
            if rt.left is None or rt.right is None:
                preds[x] = rt.prediction
                break
            elif xTe[x,rt.feature] <= rt.cut:
                rt = rt.left
            else:
                rt = rt.right

    return preds

In [ ]:
Prediction = evaltree(cart(xTr, yTr),xTe)
print(Prediction)

[2. 2. 0. 0. 2. 0. 0. 0. 2. 2. 0. 2. 2. 0. 0. 2. 2. 2. 2. 2. 2. 0. 2. 2.
 0. 2. 2. 2. 2. 2. 2. 2. 0. 2. 2. 2. 0. 0. 2. 2. 2. 2. 2. 0. 0. 2. 0. 2.
 0. 0. 0. 2. 2. 2. 0. 2. 2. 0. 2. 2. 2. 0. 0. 2. 2. 0. 0. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 0. 2. 0. 2. 0. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 0. 0. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
def accuracy(truth,preds):

    truth = truth.flatten()
    preds = preds.flatten()
    diff = np.abs(truth - preds)
    false = np.count_nonzero(diff)
    corr = diff.shape[0] - false
    accuracy = np.float64(corr / diff.shape[0])

    return accuracy

In [ ]:
result=accuracy(yTe,Prediction)
t1 = time.time()
print("You obtained %.2f%% classification acccuracy" % (result*100.0))

You obtained 83.33% classification acccuracy
